# Newton

we want to create a symulation of 2 planets. To do so we have  a domain $\Omega$ and two bodies (ball-shaped) $B_1$ and $B_2$ with masses $m_1$ and $m_2$ respectively.

In the domain there are 2 circels, one with radius $r_1$ and one with radius $r_2$. The circels are centered at $c_1$ and $c_2$ respectively. They create a linear functionals $f_1$ and $f_2$ that are $M_1$ and $M_2$ inside the circels and 0 outside.
$$
f_1 (x) = M_1\mathbb{1}_{B_1} (x) \\
f_2 (x) = M_2\mathbb{1}_{B_2} (x) $$

In particular a linear functional depending on the position of the bodies at each time step. The functionals are defined as follows:
$$
f = f_1 + f_2
$$

The position of each point $u$ is given by the following equation:
$$
\dot u = v \\   
\dot v = g  =-\nabla \phi \\
$$
And finally the potential $\phi$ is given by:
$$
\Delta \phi = f
$$


In [1]:
from ngsolve import *
from ngsolve import *
from netgen.csg import *
import scipy.sparse as sp
import matplotlib.pylab as plt



# import netgen.webgui  
# from ngsolve.webgui import Draw
import netgen.gui
%gui tk

importing NGSolve-6.2.2204
optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2204
Using Lapack
Including sparse direct solver Pardiso
Running parallel using 16 thread(s)


In [2]:
# create planet one and planet two
PL1 = Sphere((-5,0,0), 1).bc("PL_bc1").mat("planet1").maxh(0.3)
PL2 = Sphere(( 5,0,0), 1).bc("PL_bc2").mat("planet2").maxh(0.3)
Planets = PL1 + PL2
Planets.mat("Planets")
Void = Sphere((0,0,0), 10).bc("outer_bc") - PL1 - PL2
Void.mat("Void")

geo = CSGeometry()
geo.Add(Planets)
geo.Add(Void)

mesh = Mesh(geo.GenerateMesh(maxh=3, grading=0.4))
mesh.Curve(1)

Draw(mesh)

 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 3
 Optimize Surface
 Surface 2 / 3
 Optimize Surface
 Surface 3 / 3
 Optimize Surface
 Delaunay meshing
 Delaunay meshing
 start tetmeshing
 Success !
 Remove Illegal Elements
 Volume Optimization
 Curve elements, order = 1


In [16]:
RHO = L2(mesh, order=1, dirichlet="outer_bc")
U = H1(mesh, order=2, dirichlet="outer_bc")
W = VectorValued(U, 3)


r , dr = RHO.TnT()
u,  du = U.TnT()
w, dw =W.TnT()

# density of a planet
gfrho     = GridFunction(RHO)
gfrho.Set(CF((1)),definedon = mesh.Materials("Planets"),  bonus_intorder =10)
Draw (gfrho, mesh, "phi")

# displacement of a planet
gfu     = GridFunction(W)

# velocity field of a planet
gfv     = GridFunction(W)
gfv.Set(CF((0,0,IfPos(x,1,-1) )),definedon = mesh.Materials("Planets"),  bonus_intorder =10)


# gravitational potential tp be solved
gfphi     = GridFunction(U)




In [17]:
Draw(gfv,mesh,"gfv")


In [18]:
inverse = "sparsecholesky"

with TaskManager():
    ## u and v 
    mass_u = BilinearForm(W)
    mass_u += InnerProduct(w, dw)*dx
    mass_u.Assemble()
    mass_u_inv = mass_u.mat.Inverse(inverse=inverse)

    mass_rho = BilinearForm(RHO )
    mass_rho += InnerProduct(r, dr)*dx
    mass_rho.Assemble()
    mass_rho_inv = mass_rho.mat.Inverse(inverse=inverse)




    


# A = sp.csr_matrix(mass_phi.mat.CSR())
# plt.rcParams['figure.figsize'] = (10,10)
# plt.spy(A)
# plt.show()


In [19]:
OpGrad = BilinearForm (testspace=W, trialspace=U)
OpGrad += InnerProduct(grad(u),dw)*dx
OpGrad.Assemble()

# print matrix
# B = sp.csr_matrix(OpGrad.mat.CSR())
# plt.rcParams['figure.figsize'] = (10,10)
# plt.spy(B)
# plt.show()

In [20]:
laplacian = BilinearForm(U)
laplacian += grad(u)*grad(du)*dx
laplacian.Assemble()
inv_laplacian = laplacian.mat.Inverse(U.FreeDofs(), inverse="sparsecholesky")


In [21]:
dt = 0.01
with TaskManager():

    for i in range(10):
        gfu.vec.data += dt*gfv.vec
        gfv.vec.data += -dt*mass_u_inv @ OpGrad.mat * gfphi.vec
        gfphi.vec.data += dt*inv_laplacian @ mass_rho.mat * gfrho.vec
        Draw(gfu,mesh, "gfu")